In [1]:
import numpy as np
import pickle
import math
import matplotlib.pyplot as plt
import pandas as pd
import time
import pprint
import seaborn as sns
import CMR_IA as cmr
sns.set_context('paper')
# pd.set_option('display.max_columns', None)

In [2]:
# association recognition - cued recall
with open("../Data/simuS1_group1_design.pkl", 'rb') as inp:
    df_study = pickle.load(inp)
    df_test = pickle.load(inp)
df_study = df_study.loc[df_study.session < 100]
df_test = df_test.loc[df_test.session < 100]

In [3]:
df_study

,study_itemno1,study_itemno2,study_item1,study_item2,session,order,pair_idx
0,1118,147,PRAIRIE,BOMB,0,1,0
1,581,634,FLOOR,GHOST,0,0,1
2,1087,710,PLAYGROUND,HIP,0,1,2
3,1458,679,THIMBLE,HAMBURGER,0,0,3
4,745,589,ISLAND,FOLLOWER,0,0,4
...,...,...,...,...,...,...,...
4795,1214,1171,RUBY,RAZOR,99,-1,43
4796,1538,1173,UNICORN,RECEIPT,99,-1,44
4797,618,222,GALLON,CALF,99,-1,45
4798,1229,1289,SALT,SHOELACE,99,-1,46


In [4]:
df_test

,test_itemno1,test_itemno2,test_item1,test_item2,correct_ans,order,pair_idx,test,session
0,146,1542,BOLT,UTENSIL,1,0,34,1,0
1,147,1118,BOMB,PRAIRIE,1,1,0,1,0
2,1185,159,REPTILE,BOX,0,1,-1,1,0
3,358,1421,COMPASS,SWING,1,1,35,1,0
4,1201,1359,ROCK,SPRINKLE,1,0,12,1,0
...,...,...,...,...,...,...,...,...,...
5995,86,-1,BANQUET,-1,1366,1,18,2,99
5996,644,-1,GOAT,-1,62,0,7,2,99
5997,1159,-1,RAFT,-1,880,1,25,2,99
5998,325,-1,CLOCK,-1,102,1,26,2,99


### Run CMR-IA

In [5]:
s_mat = np.load('../Data/wordpools/ltp_FR_similarity_matrix.npy')

In [6]:
params = cmr.make_default_params()
params.update(
    beta_enc = 0.5,
    beta_cue = 0.5,
    beta_rec = 0,
    beta_distract = 0.1,
    beta_rec_new = 0.1, # beta for retrieved item
    beta_rec_post = 0.1,
    gamma_fc = 0.5,
    gamma_cf = 0.5,
    s_fc = 0.1,
    s_cf = 0.1,
    c_thresh = 0.01,
    c_thresh_ass = 0.70,
    a = 2800,
    b = 2.14,
    m = 0,
    n = 1,
    kappa = 0.02,
    lamb = 0,
    eta = 0.01,
    phi_s = 1,
    phi_d = 0.6,
    nitems_in_accumulator = 96,
    d_ass = 1,
    learn_while_retrieving = True,
    omega = 2,
    alpha = 1
)
params

{'beta_enc': 0.5,
 'beta_rec': 0,
 'beta_cue': 0.5,
 'beta_rec_post': 0.1,
 'beta_distract': 0.1,
 'phi_s': 1,
 'phi_d': 0.6,
 's_cf': 0.1,
 's_fc': 0.1,
 'kappa': 0.02,
 'eta': 0.01,
 'omega': 2,
 'alpha': 1,
 'c_thresh': 0.01,
 'c_thresh_ass': 0.7,
 'd_ass': 1,
 'lamb': 0,
 'rec_time_limit': 60000.0,
 'dt': 10,
 'nitems_in_accumulator': 96,
 'max_recalls': 50,
 'learn_while_retrieving': True,
 'a': 2800,
 'b': 2.14,
 'm': 0,
 'n': 1,
 'c1': 0,
 'No_recall': None,
 'var_enc': 1,
 'bad_enc_ratio': 1,
 'gamma_fc': 0.5,
 'gamma_cf': 0.5,
 'beta_rec_new': 0.1}

In [7]:
df_simu, f_in_acc, f_in_dif = cmr.run_success_multi_sess(params,df_study, df_test ,s_mat)
df_simu

CMR2 Time: 5.128671169281006


,session,test_itemno1,test_itemno2,s_resp,s_rt,csim
0,0,146,1542,1.0,585.143799,0.741550
1,0,147,1118,0.0,828.007080,0.579324
2,0,1185,159,0.0,1232.549683,0.393427
3,0,358,1421,1.0,527.316589,0.790174
4,0,1201,1359,1.0,625.207214,0.710603
...,...,...,...,...,...,...
5995,99,86,-1,1366.0,700.000000,0.776832
5996,99,644,-1,62.0,700.000000,0.679916
5997,99,1159,-1,1475.0,1230.000000,0.407420
5998,99,325,-1,102.0,820.000000,0.782163


In [8]:
df_simu = df_simu.merge(df_test,on=['session','test_itemno1','test_itemno2'])
df_simu['correct'] = df_simu.s_resp == df_simu.correct_ans
df_simu

,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test_item1,test_item2,correct_ans,order,pair_idx,test,correct
0,0,146,1542,1.0,585.143799,0.741550,BOLT,UTENSIL,1,0,34,1,True
1,0,147,1118,0.0,828.007080,0.579324,BOMB,PRAIRIE,1,1,0,1,False
2,0,1185,159,0.0,1232.549683,0.393427,REPTILE,BOX,0,1,-1,1,True
3,0,358,1421,1.0,527.316589,0.790174,COMPASS,SWING,1,1,35,1,True
4,0,1201,1359,1.0,625.207214,0.710603,ROCK,SPRINKLE,1,0,12,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,99,86,-1,1366.0,700.000000,0.776832,BANQUET,-1,1366,1,18,2,True
5996,99,644,-1,62.0,700.000000,0.679916,GOAT,-1,62,0,7,2,True
5997,99,1159,-1,1475.0,1230.000000,0.407420,RAFT,-1,880,1,25,2,False
5998,99,325,-1,102.0,820.000000,0.782163,CLOCK,-1,102,1,26,2,True


In [9]:
# get f_in
sessions = np.unique(df_simu.session)
tmp_corr_fin = []
tmp_omax_fin = []
test1_num = 40

for sess in sessions:
    df_tmp = df_study.loc[df_study.session == sess]
    tmp1 = df_tmp.study_itemno1.to_numpy()
    tmp2 = df_tmp.study_itemno2.to_numpy()
    tmp = np.concatenate((tmp1, tmp2))
    tmp = np.sort(tmp)

    tmp_corr = df_simu.loc[df_simu.session == sess,"correct_ans"][test1_num:]
    corrid = np.searchsorted(tmp, tmp_corr)

    corr_fin = [f_in_dif[sess][i][id] for i, id in enumerate(corrid)]
    omax_fin = [np.max(np.delete(f_in_dif[sess][i], id)) for i, id in enumerate(corrid)]

    tmp_corr_fin = tmp_corr_fin + [-1] * test1_num + corr_fin
    tmp_omax_fin = tmp_omax_fin + [-1] * test1_num + omax_fin

df_simu['corr_fin'] = tmp_corr_fin
df_simu['omax_fin'] = tmp_omax_fin
df_simu

,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test_item1,test_item2,correct_ans,order,pair_idx,test,correct,corr_fin,omax_fin
0,0,146,1542,1.0,585.143799,0.741550,BOLT,UTENSIL,1,0,34,1,True,-1.000000,-1.000000
1,0,147,1118,0.0,828.007080,0.579324,BOMB,PRAIRIE,1,1,0,1,False,-1.000000,-1.000000
2,0,1185,159,0.0,1232.549683,0.393427,REPTILE,BOX,0,1,-1,1,True,-1.000000,-1.000000
3,0,358,1421,1.0,527.316589,0.790174,COMPASS,SWING,1,1,35,1,True,-1.000000,-1.000000
4,0,1201,1359,1.0,625.207214,0.710603,ROCK,SPRINKLE,1,0,12,1,True,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,99,86,-1,1366.0,700.000000,0.776832,BANQUET,-1,1366,1,18,2,True,0.327093,-0.238325
5996,99,644,-1,62.0,700.000000,0.679916,GOAT,-1,62,0,7,2,True,0.171727,-0.265120
5997,99,1159,-1,1475.0,1230.000000,0.407420,RAFT,-1,880,1,25,2,False,-1.309945,-0.143163
5998,99,325,-1,102.0,820.000000,0.782163,CLOCK,-1,102,1,26,2,True,0.337418,-0.277479


In [10]:
# recognition performance
df_recog = df_simu.query("test==1")
df_recog.groupby("correct_ans")["s_resp"].mean().to_frame(name="Yes rate")

,Yes rate
correct_ans,
0,0.0455
1,0.8615


In [11]:
# cued recall performance
df_cr = df_simu.query("test==2")
df_cr.correct.mean()

0.754

In [12]:
def get_pair(df_tmp):
    df_tmp_pair = pd.pivot_table(df_tmp,index="pair_idx",columns="test",values="correct")
    df_tmp_pair.columns = ["test1","test2"]
    df_tmp_pair.reset_index(inplace=True)
    return df_tmp_pair

df_simu_p = df_simu.query("pair_idx >= 0")
df_pair = df_simu_p.groupby("session").apply(get_pair).reset_index()
df_pair

,session,level_1,pair_idx,test1,test2
0,0,0,0,0.0,1.0
1,0,1,2,0.0,0.0
2,0,2,3,1.0,1.0
3,0,3,4,1.0,1.0
4,0,4,6,1.0,1.0
...,...,...,...,...,...
1995,99,15,28,1.0,1.0
1996,99,16,29,1.0,1.0
1997,99,17,32,0.0,0.0
1998,99,18,38,1.0,1.0


In [13]:
df_tab = pd.crosstab(index=df_pair.test2,columns=df_pair.test1,normalize='all')
df_tab

test1,0.0,1.0
test2,,
0.0,0.1220,0.1240
1.0,0.0165,0.7375


In [14]:
def Yule_Q(A, B, C, D):
    return (A * D - B * C) / (A * D + B * C)
Yule_Q(df_tab[1][1],df_tab[0][1],df_tab[1][0],df_tab[0][0])

0.9555318894600144

In [15]:
# add 0.5 to each cell
Yule_Q(df_tab[1][1]+0.5,df_tab[0][1]+0.5,df_tab[1][0]+0.5,df_tab[0][0]+0.5)

0.4097256371443407